# DinoJourney Logo Generation

This Jupyter Notebook was used to generate the images used in our ShellHacks 2024 Project

In [2]:
# dependencies
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import replicate
import time
import json
from PIL import Image as PilImage, ImageDraw, ImageFont
import requests
from io import BytesIO
import os
from getpass import getpass
import matplotlib.pyplot as plt
import httpx
import openai

## Replicate

In [3]:
# get your API Key from Replicate website and paste it here.
REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

 ········


In [24]:
def replicate_run(model, prompt, num_images):
    title = model.split(':')[0]

    # Set the API key using environment variable
    api_token = os.getenv("REPLICATE_API_TOKEN")  # Ensure API key is set
    if not api_token:
        raise ValueError("API token not found or is empty")

    headers = {
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json"
    }

    # Define the payload (you may need to adjust this based on the model's input schema)
    data = {
        "version": model.split(":")[1],
        "input": {
            "prompt": prompt,
            "num_outputs": num_images
            # Add more parameters here depending on the model's schema (e.g., guidance_scale, controlnet, etc.)
        }
    }

    # Measure time taken for API call
    start_time = time.time()

    try:
        # Make the POST request to start the prediction
        response = httpx.post("https://api.replicate.com/v1/predictions", headers=headers, json=data)
        response.raise_for_status()  # Raise an exception for HTTP errors
        response_json = response.json()
    except httpx.HTTPStatusError as e:
        print(f"HTTP error occurred: {e.response.status_code} - {e.response.text}")
        return None, title, 0, 0
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None, title, 0, 0
    
    # Extract the prediction URL for polling
    prediction_url = response_json.get("urls", {}).get("get")
    if not prediction_url:
        raise ValueError("Could not retrieve prediction URL")

    print(f"Prediction started, polling at: {prediction_url}")

    # Poll the result until the status is 'succeeded'
    def poll_prediction(url, headers, interval=5):
        while True:
            response = httpx.get(url, headers=headers)
            result = response.json()

            # Check if the status is complete
            status = result.get("status")
            if status == "succeeded":
                return result.get("output")
            elif status == "failed":
                raise Exception("Prediction failed")

            # Print the status every poll
            print(f"Prediction status: {status}, retrying in {interval} seconds...")
            time.sleep(interval)

    # Poll for the output dynamically
    try:
        output = poll_prediction(prediction_url, headers)
    except Exception as e:
        print(f"Error during polling: {str(e)}")
        return None, title, 0, 0

    end_time = time.time()
    total_time = end_time - start_time
    average_time = total_time / num_images

    return output, title, total_time, average_time

Enter your prompt below and run the cell! Note it might take some time for all images to generate.

### Experimenting with multiple text-to-image-models

Research work to be presented next month at HCOMP shows that side-by-side text-to-image AI models can help users identify model styles more easily. 

We take advantage of this by generating images from **different models** using the **same prompt,** allowing us to narrow down which models we would like to proceed with.

In [28]:
def display_4x4_grid(outputs):
    fig, axs = plt.subplots(4, 4, figsize=(12, 12))
    fig.suptitle("4x4 Model Output Gallery", fontsize=16)

    for i, (model_output, title, total_time, average_time) in enumerate(outputs[:16]):
        # Select the first image for each model
        image_url = model_output[0]
        response = requests.get(image_url)
        img = PilImage.open(BytesIO(response.content))

        axs[i//4, i%4].imshow(img)
        axs[i//4, i%4].set_title(f"{title[:]}\n{total_time:.2f}s")
        axs[i//4, i%4].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
# 19 other models that you can pick from
outputs = []
prompt, num = "A cute green magnifying glass", 1

outputs.append(replicate_run("lucataco/ssd-1b:b19e3639452c59ce8295b82aba70a231404cb062f2eb580ea894b31e8ce5bbb6", prompt, num))
outputs.append(replicate_run("playgroundai/playground-v2.5-1024px-aesthetic:a45f82a1382bed5c7aeb861dac7c7d191b0fdf74d8d57c4a0e6ed7d4d0bf7d24", prompt, num))
outputs.append(replicate_run("fofr/realvisxl-v3-multi-controlnet-lora:90a4a3604cd637cb9f1a2bdae1cfa9ed869362ca028814cdce310a78e27daade", prompt, num))
outputs.append(replicate_run("fofr/sticker-maker:4acb778eb059772225ec213948f0660867b2e03f277448f18cf1800b96a65a1a", prompt, num))

outputs.append(replicate_run("lucataco/realvisxl2-lcm:479633443fc6588e1e8ae764b79cdb3702d0c196e0cb2de6db39ce577383be77", prompt, num))
outputs.append(replicate_run("playgroundai/playground-v2-1024px-aesthetic:42fe626e41cc811eaf02c94b892774839268ce1994ea778eba97103fe1ef51b8", prompt, num))
outputs.append(replicate_run("lucataco/realvisxl-v2.0:7d6a2f9c4754477b12c14ed2a58f89bb85128edcdd581d24ce58b6926029de08", prompt, num))
outputs.append(replicate_run("fofr/sdxl-multi-controlnet-lora:89eb212b3d1366a83e949c12a4b45dfe6b6b313b594cb8268e864931ac9ffb16", prompt, num))

outputs.append(replicate_run("lucataco/dreamshaper-xl-turbo:0a1710e0187b01a255302738ca0158ff02a22f4638679533e111082f9dd1b615", prompt, num))
outputs.append(replicate_run("lucataco/open-dalle-v1.1:1c7d4c8dec39c7306df7794b28419078cb9d18b9213ab1c21fdc46a1deca0144", prompt, num))
outputs.append(replicate_run("ai-forever/kandinsky-2-1:a768f3c2e174c54b576cc4f222e789e161160403d0cd0ace41eeb9a0f8c8d5f8", prompt, num))
outputs.append(replicate_run("adirik/realvisxl-v3.0-turbo:3dc73c805b11b4b01a60555e532fd3ab3f0e60d26f6584d9b8ba7e1b95858243", prompt, num))

outputs.append(replicate_run("lucataco/pixart-xl-2:816c99673841b9448bc2539834c16d40e0315bbf92fef0317b57a226727409bb", prompt, num))
outputs.append(replicate_run("adirik/realvisxl-v4.0:85a58cc71587cc27539b7c83eb1ce4aea02feedfb9a9fae0598cebc110a3d695", prompt, num))
outputs.append(replicate_run("lucataco/proteus-v0.3:b28b79d725c8548b173b6a19ff9bffd16b9b80df5b18b8dc5cb9e1ee471bfa48", prompt, num))
outputs.append(replicate_run("lucataco/thinkdiffusionxl:c41c12756b561bc81047a9307c9143589d158ef084655dbb3073b4f08ff54f32", prompt, num))

outputs.append(replicate_run("artificialguybr/nebul.redmond:1abd2490609ffab31652791c065fa2da180053b77fe0ed0e7e879460bf549d33", prompt, num))
outputs.append(replicate_run("fofr/txt2img:18f1d88376233fef85e7289d65326b9ceabd3a78215a6833bfb7775792f42b79", prompt, num))
outputs.append(replicate_run("lucataco/sdxl-deepcache:eaf678fb34006669e9a3c6dd5971e2279bf20ee0adeced464d7b6d95de16dc93", prompt, num))

display_4x4_grid(outputs[-3]) # not displaying the last 3 models

### Experimenting with a single text-to-image model

After running the initial analysis above, "bytedance/sdxl-lightining-4step" was chosen due to its speed and quality.

Additionally, its generated images aligned closely with what our team envisioned when we prompted for a "simplistic and cartoonish" image.

Now that we have one model we will be working with, we will try different prompts to gauge which prompt produces the best image for our needs.

In [54]:
# stored generated images
outputs = []

# note that number of generated images per model must be <= 4
prompt, num = "A cute and happy green magnifying glass", 4
outputs.append(replicate_run("bytedance/sdxl-lightning-4step:727e49a643e999d602a896c774a0658ffefea21465756a6ce24b7ea4165eba6a", prompt, num))

prompt, num = "A green magnifying glass", 4
outputs.append(replicate_run("bytedance/sdxl-lightning-4step:727e49a643e999d602a896c774a0658ffefea21465756a6ce24b7ea4165eba6a", prompt, num))

prompt, num = "A nice green magnifying glass", 4
outputs.append(replicate_run("bytedance/sdxl-lightning-4step:727e49a643e999d602a896c774a0658ffefea21465756a6ce24b7ea4165eba6a", prompt, num))

prompt, num = "An awesome green magnifying glass", 4
outputs.append(replicate_run("bytedance/sdxl-lightning-4step:727e49a643e999d602a896c774a0658ffefea21465756a6ce24b7ea4165eba6a", prompt, num))

prompt, num = "A happy little cartoonish man", 4
outputs.append(replicate_run("bytedance/sdxl-lightning-4step:727e49a643e999d602a896c774a0658ffefea21465756a6ce24b7ea4165eba6a", prompt, num))

Prediction started, polling at: https://api.replicate.com/v1/predictions/1jycgfyznhrj20cj6wgsy7eqfg
Prediction status: starting, retrying in 5 seconds...
Prediction started, polling at: https://api.replicate.com/v1/predictions/15cem47ws9rj60cj6wgthpj6cc
Prediction status: processing, retrying in 5 seconds...
Prediction started, polling at: https://api.replicate.com/v1/predictions/qkg5wz8t71rj60cj6whbs3be7r
Prediction status: processing, retrying in 5 seconds...
Prediction started, polling at: https://api.replicate.com/v1/predictions/k7wnr1stjxrj00cj6whaxz7d80
Prediction status: processing, retrying in 5 seconds...
Prediction started, polling at: https://api.replicate.com/v1/predictions/80n1hbjt85rj00cj6wh8418axg
Prediction status: processing, retrying in 5 seconds...
Prediction started, polling at: https://api.replicate.com/v1/predictions/mg1hqd3pyhrj40cj6wh8v37pvw
Prediction status: processing, retrying in 5 seconds...


## OPENAI

In [ ]:
# get your API Key from Replicate website and paste it here.
OPEN_AI_KEY = getpass()
os.environ["OPEN_AI_KEY"] = OPEN_AI_KEY

response = openai.Image.create(
    prompt="A cute happy magnifying glass cartoonish",
    n=1,
    size="256x256"
)

# The response includes a list of generated image URLs
image_url = response["data"][0]["url"]
print(image_url)